In [ ]:
%%sql
-- Write your SQL query here:

-- Your approach: Explain your thinking before writing the code
-- 1. Join orders with order_items to get detailed item data
-- 2. Join with products to get category information  
-- 3. Filter for last 30 days
-- 4. Group by category and aggregate

-- YOUR CODE HERE
SELECT 
    -- Fill in your solution
FROM 
    -- Your tables and joins
WHERE 
    -- Your filter conditions
GROUP BY 
    -- Your grouping
ORDER BY 
    -- Your ordering


In [ ]:
%%sql
-- SOLUTION 1: Standard approach with explicit joins
SELECT 
    p.category,
    COUNT(DISTINCT o.order_id) AS total_orders,
    SUM(oi.line_total) AS total_revenue,
    ROUND(SUM(oi.line_total) / COUNT(DISTINCT o.order_id), 2) AS avg_order_value
FROM 
    orders o
    INNER JOIN order_items oi ON o.order_id = oi.order_id
    INNER JOIN products p ON oi.product_id = p.product_id
WHERE 
    o.order_date >= CURRENT_DATE - INTERVAL '30 days'
    AND o.order_date < CURRENT_DATE
GROUP BY 
    p.category
ORDER BY 
    total_revenue DESC;

-- ALTERNATIVE SOLUTION 2: Using CTE for clarity
WITH recent_orders AS (
    SELECT o.order_id, o.customer_id, o.order_date, o.total_amount
    FROM orders o
    WHERE o.order_date >= CURRENT_DATE - INTERVAL '30 days'
),
order_details AS (
    SELECT 
        ro.order_id,
        p.category,
        oi.line_total
    FROM recent_orders ro
    JOIN order_items oi ON ro.order_id = oi.order_id
    JOIN products p ON oi.product_id = p.product_id
)
SELECT 
    category,
    COUNT(DISTINCT order_id) AS total_orders,
    SUM(line_total) AS total_revenue,
    ROUND(SUM(line_total) / COUNT(DISTINCT order_id), 2) AS avg_order_value
FROM order_details
GROUP BY category
ORDER BY total_revenue DESC;

-- ALTERNATIVE SOLUTION 3: Window function approach for additional insights
SELECT 
    p.category,
    COUNT(DISTINCT o.order_id) AS total_orders,
    SUM(oi.line_total) AS total_revenue,
    ROUND(SUM(oi.line_total) / COUNT(DISTINCT o.order_id), 2) AS avg_order_value,
    ROUND(100.0 * SUM(oi.line_total) / SUM(SUM(oi.line_total)) OVER(), 2) AS revenue_percentage
FROM 
    orders o
    INNER JOIN order_items oi ON o.order_id = oi.order_id
    INNER JOIN products p ON oi.product_id = p.product_id
WHERE 
    o.order_date >= CURRENT_DATE - INTERVAL '30 days'
GROUP BY 
    p.category
ORDER BY 
    total_revenue DESC;


In [ ]:
%%sql
-- Write your SQL query here:

-- Your approach: Think through the steps
-- 1. Get order counts by fulfillment center and state
-- 2. Rank centers within each state  
-- 3. Keep only top 3 per state
-- 4. Use string aggregation to concatenate names
-- 5. Sum the order counts for the top 3

-- YOUR CODE HERE
WITH fc_order_counts AS (
    -- Step 1: Count orders by fulfillment center
    SELECT 
        -- Your aggregation logic here
    FROM 
        -- Your joins here
    GROUP BY 
        -- Your grouping here
),
ranked_centers AS (
    -- Step 2: Rank centers within each state
    SELECT 
        -- Your ranking logic here
    FROM fc_order_counts
),
top_3_centers AS (
    -- Step 3: Filter to top 3
    SELECT * 
    FROM ranked_centers 
    WHERE -- Your ranking filter here
)
-- Step 4 & 5: Aggregate the results
SELECT 
    -- Your final aggregation with STRING_AGG/LISTAGG
FROM top_3_centers
GROUP BY -- Your final grouping
ORDER BY -- Your ordering


In [ ]:
%%sql
-- SOLUTION 1: Using ROW_NUMBER() and STRING_AGG (PostgreSQL/SQL Server)
WITH fc_order_counts AS (
    SELECT 
        fc.state,
        fc.fc_name,
        COUNT(o.order_id) AS order_count
    FROM 
        fulfillment_centers fc
        LEFT JOIN orders o ON fc.fc_id = o.fulfillment_center_id
    WHERE 
        o.order_date >= CURRENT_DATE - INTERVAL '30 days'  -- Optional: filter recent orders
    GROUP BY 
        fc.state, fc.fc_name, fc.fc_id
),
ranked_centers AS (
    SELECT 
        state,
        fc_name,
        order_count,
        ROW_NUMBER() OVER (PARTITION BY state ORDER BY order_count DESC, fc_name) AS rn
    FROM fc_order_counts
),
top_3_centers AS (
    SELECT * 
    FROM ranked_centers 
    WHERE rn <= 3
)
SELECT 
    state,
    STRING_AGG(fc_name, ', ' ORDER BY rn) AS top_3_centers,
    SUM(order_count) AS total_orders_top3
FROM top_3_centers
GROUP BY state
ORDER BY total_orders_top3 DESC;

-- SOLUTION 2: Using RANK() instead of ROW_NUMBER() (handles ties differently)
WITH fc_order_counts AS (
    SELECT 
        fc.state,
        fc.fc_name,
        COUNT(o.order_id) AS order_count
    FROM 
        fulfillment_centers fc
        LEFT JOIN orders o ON fc.fc_id = o.fulfillment_center_id
    GROUP BY 
        fc.state, fc.fc_name, fc.fc_id
),
ranked_centers AS (
    SELECT 
        state,
        fc_name,
        order_count,
        RANK() OVER (PARTITION BY state ORDER BY order_count DESC) AS rnk
    FROM fc_order_counts
),
top_3_centers AS (
    SELECT * 
    FROM ranked_centers 
    WHERE rnk <= 3
)
SELECT 
    state,
    STRING_AGG(fc_name || ' (' || order_count || ')', ', ' ORDER BY rnk, fc_name) AS top_3_centers_with_counts,
    SUM(order_count) AS total_orders_top3
FROM top_3_centers
GROUP BY state
ORDER BY total_orders_top3 DESC;

-- SOLUTION 3: Oracle/MySQL compatible using LISTAGG/GROUP_CONCAT
WITH fc_order_counts AS (
    SELECT 
        fc.state,
        fc.fc_name,
        COUNT(o.order_id) AS order_count
    FROM 
        fulfillment_centers fc
        LEFT JOIN orders o ON fc.fc_id = o.fulfillment_center_id
    GROUP BY 
        fc.state, fc.fc_name, fc.fc_id
),
ranked_centers AS (
    SELECT 
        state,
        fc_name,
        order_count,
        ROW_NUMBER() OVER (PARTITION BY state ORDER BY order_count DESC, fc_name) AS rn
    FROM fc_order_counts
),
top_3_centers AS (
    SELECT * 
    FROM ranked_centers 
    WHERE rn <= 3
)
SELECT 
    state,
    -- Oracle: LISTAGG(fc_name, ', ') WITHIN GROUP (ORDER BY rn) AS top_3_centers,
    -- MySQL: GROUP_CONCAT(fc_name ORDER BY rn SEPARATOR ', ') AS top_3_centers,
    SUM(order_count) AS total_orders_top3
FROM top_3_centers
GROUP BY state
ORDER BY total_orders_top3 DESC;


In [ ]:
%%sql
-- Your approach: Think through the date logic
-- 1. Generate months for the past year
-- 2. Find the last day of each month
-- 3. Calculate what day of week that is
-- 4. Subtract days to get to the last Saturday

-- YOUR CODE HERE
WITH months_range AS (
    -- Generate the past 12 months
    SELECT -- Your month generation logic
),
last_day_of_month AS (
    -- Find last day of each month
    SELECT -- Your last day calculation  
),
last_saturday AS (
    -- Calculate last Saturday of each month
    SELECT -- Your Saturday calculation
)
SELECT * FROM last_saturday
ORDER BY month_date;


In [ ]:
%%sql
-- SOLUTION 1: PostgreSQL approach with generate_series
WITH months_range AS (
    SELECT 
        DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 month' * generate_series(0, 11)) AS month_start
),
last_day_of_month AS (
    SELECT 
        month_start,
        (month_start + INTERVAL '1 month' - INTERVAL '1 day')::DATE AS month_end
    FROM months_range
),
last_saturday AS (
    SELECT 
        month_start,
        month_end,
        -- Saturday is day 6 (0=Sunday, 6=Saturday)
        -- Subtract days to get to last Saturday: (day_of_week + 1) % 7
        month_end - INTERVAL '1 day' * ((EXTRACT(DOW FROM month_end) + 1) % 7) AS last_saturday_date
    FROM last_day_of_month
)
SELECT 
    TO_CHAR(month_start, 'YYYY-MM') AS month,
    last_saturday_date,
    TO_CHAR(last_saturday_date, 'Day') AS day_name
FROM last_saturday
ORDER BY month_start DESC;

-- SOLUTION 2: SQL Server approach
WITH months_range AS (
    SELECT 
        DATEFROMPARTS(YEAR(DATEADD(month, -number, GETDATE())), 
                      MONTH(DATEADD(month, -number, GETDATE())), 1) AS month_start
    FROM master.dbo.spt_values 
    WHERE type = 'P' AND number BETWEEN 0 AND 11
),
last_saturday AS (
    SELECT 
        month_start,
        EOMONTH(month_start) AS month_end,
        DATEADD(day, -(DATEPART(weekday, EOMONTH(month_start)) % 7), EOMONTH(month_start)) AS last_saturday_date
    FROM months_range
)
SELECT 
    FORMAT(month_start, 'yyyy-MM') AS month,
    last_saturday_date,
    DATENAME(weekday, last_saturday_date) AS day_name
FROM last_saturday
ORDER BY month_start DESC;

-- SOLUTION 3: MySQL approach  
WITH RECURSIVE months_range AS (
    SELECT 
        LAST_DAY(CURDATE() - INTERVAL 1 MONTH) AS month_end,
        0 as month_offset
    UNION ALL
    SELECT 
        LAST_DAY(CURDATE() - INTERVAL (month_offset + 1) MONTH),
        month_offset + 1
    FROM months_range 
    WHERE month_offset < 11
)
SELECT 
    DATE_FORMAT(month_end, '%Y-%m') AS month,
    DATE_SUB(month_end, INTERVAL (WEEKDAY(month_end) + 2) % 7 DAY) AS last_saturday_date,
    DAYNAME(DATE_SUB(month_end, INTERVAL (WEEKDAY(month_end) + 2) % 7 DAY)) AS day_name
FROM months_range
ORDER BY month_end DESC;

-- SOLUTION 4: Oracle approach
WITH months_range AS (
    SELECT 
        ADD_MONTHS(TRUNC(SYSDATE, 'MM'), -LEVEL) AS month_start
    FROM dual
    CONNECT BY LEVEL <= 12
),
last_saturday AS (
    SELECT 
        month_start,
        LAST_DAY(month_start) AS month_end,
        LAST_DAY(month_start) - MOD(TO_CHAR(LAST_DAY(month_start), 'D') - 1, 7) AS last_saturday_date
    FROM months_range
)
SELECT 
    TO_CHAR(month_start, 'YYYY-MM') AS month,
    last_saturday_date,
    TO_CHAR(last_saturday_date, 'Day') AS day_name
FROM last_saturday
ORDER BY month_start DESC;


In [ ]:
%%sql
-- Your approach: Calculate days of supply
-- 1. For each day, calculate days_of_supply = supply / demand
-- 2. Handle edge cases (zero demand, negative values)
-- 3. Categorize supply levels

SELECT 
    date_col,
    supply AS current_supply,
    demand AS daily_demand,
    CASE 
        WHEN demand = 0 THEN -- Handle division by zero
        WHEN demand > 0 THEN -- Calculate days of supply
        ELSE -- Handle negative demand
    END AS days_of_supply,
    CASE 
        WHEN -- Your status logic here
        WHEN -- Your status logic here  
        ELSE 'Normal'
    END AS supply_status
FROM inventory_daily
ORDER BY date_col;


In [ ]:
%%sql
-- SOLUTION: Days of Supply Calculation
SELECT 
    date_col,
    supply AS current_supply,
    demand AS daily_demand,
    CASE 
        WHEN demand = 0 THEN 999  -- Infinite supply if no demand
        WHEN demand > 0 THEN ROUND(supply::DECIMAL / demand, 2)
        ELSE 0  -- No supply if negative demand
    END AS days_of_supply,
    CASE 
        WHEN demand = 0 THEN 'Excess'
        WHEN supply::DECIMAL / NULLIF(demand, 0) < 2 THEN 'Critical'
        WHEN supply::DECIMAL / NULLIF(demand, 0) < 7 THEN 'Low'
        ELSE 'Normal'
    END AS supply_status
FROM inventory_daily
ORDER BY date_col;

-- ALTERNATIVE: With rolling average demand for smoother calculation
WITH rolling_demand AS (
    SELECT 
        date_col,
        supply,
        demand,
        AVG(demand) OVER (
            ORDER BY date_col 
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ) AS avg_demand_3day
    FROM inventory_daily
)
SELECT 
    date_col,
    supply AS current_supply,
    demand AS daily_demand,
    ROUND(avg_demand_3day, 1) AS avg_demand_3day,
    CASE 
        WHEN avg_demand_3day = 0 THEN 999
        WHEN avg_demand_3day > 0 THEN ROUND(supply / avg_demand_3day, 2)
        ELSE 0
    END AS days_of_supply_smoothed,
    CASE 
        WHEN avg_demand_3day = 0 THEN 'Excess'
        WHEN supply / NULLIF(avg_demand_3day, 0) < 2 THEN 'Critical'
        WHEN supply / NULLIF(avg_demand_3day, 0) < 7 THEN 'Low'
        ELSE 'Normal'
    END AS supply_status
FROM rolling_demand
ORDER BY date_col;


In [ ]:
%%sql
-- SOLUTION 1: Using DENSE_RANK (handles ties, keeps positions consecutive)
WITH salary_ranks AS (
    SELECT 
        salary,
        DENSE_RANK() OVER (ORDER BY salary DESC) as rank
    FROM employees
)
SELECT salary as third_highest_salary
FROM salary_ranks 
WHERE rank = 3;

-- SOLUTION 2: Using ROW_NUMBER (ignores ties, skips positions)
WITH salary_ranks AS (
    SELECT 
        salary,
        ROW_NUMBER() OVER (ORDER BY salary DESC) as rn
    FROM (SELECT DISTINCT salary FROM employees) distinct_salaries
)
SELECT salary as third_highest_salary
FROM salary_ranks 
WHERE rn = 3;

-- SOLUTION 3: Using LIMIT/OFFSET (PostgreSQL/MySQL)
SELECT DISTINCT salary as third_highest_salary
FROM employees 
ORDER BY salary DESC
LIMIT 1 OFFSET 2;

-- SOLUTION 4: Using subqueries (works on all databases)
SELECT salary as third_highest_salary
FROM employees e1
WHERE 2 = (
    SELECT COUNT(DISTINCT e2.salary)
    FROM employees e2
    WHERE e2.salary > e1.salary
);

-- SOLUTION 5: Using TOP/ranking (SQL Server)
SELECT salary as third_highest_salary
FROM (
    SELECT DISTINCT salary,
           DENSE_RANK() OVER (ORDER BY salary DESC) as rank
    FROM employees
) ranked
WHERE rank = 3;

-- ROBUST SOLUTION: Handle case where third highest doesn't exist
WITH salary_ranks AS (
    SELECT 
        salary,
        DENSE_RANK() OVER (ORDER BY salary DESC) as rank
    FROM employees
),
third_highest AS (
    SELECT salary as third_highest_salary
    FROM salary_ranks 
    WHERE rank = 3
)
SELECT 
    CASE 
        WHEN COUNT(*) > 0 THEN MAX(third_highest_salary)
        ELSE NULL 
    END as third_highest_salary
FROM third_highest;


In [ ]:
%%sql
-- Example: Before and After Optimization

-- BEFORE: Inefficient query with multiple JOINs
SELECT 
    c.customer_name,
    o.order_date,
    p.product_name,
    oi.quantity,
    cat.category_name
FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id  
    JOIN products p ON oi.product_id = p.product_id
    JOIN categories cat ON p.category_id = cat.category_id
WHERE o.order_date >= '2023-01-01'
    AND cat.category_name = 'Electronics'
ORDER BY o.order_date DESC;

-- AFTER: Optimized with filtering and indexing strategy
-- 1. Filter early in subquery
-- 2. Use EXISTS instead of JOIN when possible
-- 3. Add proper indexes

-- Create indexes first:
-- CREATE INDEX idx_orders_date_customer ON orders(order_date, customer_id);
-- CREATE INDEX idx_products_category ON products(category_id, product_id);
-- CREATE INDEX idx_order_items_composite ON order_items(order_id, product_id);

WITH recent_electronics_orders AS (
    -- Filter orders and categories early
    SELECT DISTINCT o.order_id, o.customer_id, o.order_date
    FROM orders o
    WHERE o.order_date >= '2023-01-01'
      AND EXISTS (
          SELECT 1 FROM order_items oi 
          JOIN products p ON oi.product_id = p.product_id
          JOIN categories cat ON p.category_id = cat.category_id
          WHERE oi.order_id = o.order_id 
            AND cat.category_name = 'Electronics'
      )
)
SELECT 
    c.customer_name,
    reo.order_date,
    p.product_name,
    oi.quantity
FROM recent_electronics_orders reo
    JOIN customers c ON reo.customer_id = c.customer_id
    JOIN order_items oi ON reo.order_id = oi.order_id
    JOIN products p ON oi.product_id = p.product_id
    JOIN categories cat ON p.category_id = cat.category_id
WHERE cat.category_name = 'Electronics'
ORDER BY reo.order_date DESC;

-- PAGINATION for large result sets
SELECT 
    c.customer_name,
    o.order_date,
    p.product_name,
    oi.quantity
FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id  
    JOIN products p ON oi.product_id = p.product_id
    JOIN categories cat ON p.category_id = cat.category_id
WHERE o.order_date >= '2023-01-01'
    AND cat.category_name = 'Electronics'
ORDER BY o.order_date DESC, o.order_id DESC  -- Ensure deterministic ordering
LIMIT 100 OFFSET 0;  -- First page
